# Sprawozdanie 1
## Dzielenie kanałów, różne systemy kolorów i możliwe ich wykorzystania, histogram - rozciąganie i wyrównanie

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

### Wczytywanie przykładowego obrazka i przenoszenie go do skali szarości przy pomocy OpenCV.

In [ ]:
image = cv2.imread('lab2.jpg')

rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(rgb_image)

In [ ]:
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

plt.imshow(gray_image, cmap='gray')

In [ ]:
(r, g, b) = cv2.split(rgb_image)

gray_image_naive = r / 3 + g / 3 + b /3
plt.imshow(gray_image_naive, cmap='gray')

Metoda prostej średniej z jednakowymi wagami dla każdego koloru (czerwony, zielony, niebieski) daje nieco spaczoną konwersję do skali szarości, ponieważ ludzkie oko nie postrzega każdego ze składowych kolorów w takiej samej ilości. W wyniku badań ostatecznie określono, że podział udziału każdego bazwego koloru jaki ludzkie oczy są w stanie zauważyć wynosi: 30% dla czerwonego, 59% dla zielonego i 11% dla niebieskiego.

### Dzielenie obrazu na kolory składowe i prezentacja każdgo jako intensywność w skali szarości.

In [ ]:
(r, g, b) = cv2.split(rgb_image)

plt.title('Czerwony')
plt.imshow(r, cmap='gray')

In [ ]:
plt.title('Zielony')
plt.imshow(g, cmap='gray')

In [ ]:
plt.title('Niebieski')
plt.imshow(b, cmap='gray')

Konwersja obrazu do przestrzeni HSV i prezentacja każdego kanału w skali szarości.

In [ ]:
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

(h, s, v) = cv2.split(hsv_image)

plt.title('Hue')
plt.imshow(h, cmap='gray')

In [ ]:
plt.title('Saturation')
plt.imshow(s, cmap='gray')

In [ ]:
plt.title('Value')
plt.imshow(v, cmap='gray')

### Wycinanie wszystkich elementów obrazu poza żółtym samochodem przy pomocy progowania w skali HSV.

In [ ]:
mask_h = cv2.inRange(h, 20, 40)
mask_s = cv2.inRange(s, 120, 255)
mask_v = cv2.inRange(v, 180, 255)

masked_image = cv2.bitwise_and(hsv_image, hsv_image, mask=mask_h)
masked_image = cv2.bitwise_and(masked_image, masked_image, mask=mask_s)
masked_image = cv2.bitwise_and(masked_image, masked_image, mask=mask_v)
masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_HSV2RGB)
plt.imshow(masked_image_rgb)

Przestrzeń HSV jest wykorzystana, ponieważ pozwala przy pomocy jednego parametru - Hue - wybrać interesujący nas odcień koloru. W tym przypadku jest to okolica wartości 60 stopni, ale jako że dane w OpenCV są przechowywane w zmiennych 8-bitowych, skala parametru H jest zmniejszona do 180 stopni, więc interesująca nas wartość to okolice H ~ 30. Dodatkowo na obrazie są zastosowane progowania na pozostałych zmiennych, by jak najlepiej wydobyć żółty samochód.

### Histogram obrazu.

In [ ]:
def plot_histogram_from_rgb_image(rgb_image):
    (r, g, b) = cv2.split(rgb_image)
    counts, bins = np.histogram(r, bins=256)
    plt.scatter(bins[:-1], counts, color='r', label='r', marker='.')

    counts, bins = np.histogram(g, bins=256)
    plt.scatter(bins[:-1], counts, color='g', label='g', marker='.')

    counts, bins = np.histogram(b, bins=256)
    plt.scatter(bins[:-1], counts, color='b', label='b', marker='.')
    plt.legend()

plot_histogram_from_rgb_image(rgb_image)

### Rozciąganie kontrastu i wyrównywanie histogramu.

In [ ]:
def stretch_contrast_of_rgb_image(rgb_image):
    (r, g, b) = cv2.split(rgb_image)
    min_color = np.min(np.vstack((r, g, b)))
    max_color = np.max(np.vstack((r, g, b)))

    # have to convert to int as this operation results in floats which can be incorrectly converted (values too high)
    stretched_r = ((r - min_color) / (max_color - min_color)) * 255
    stretched_g = ((g - min_color) / (max_color - min_color)) * 255
    stretched_b = ((b - min_color) / (max_color - min_color)) * 255

    return cv2.merge((stretched_r.astype(np.uint8), stretched_g.astype(np.uint8), stretched_b.astype(np.uint8)))

contrast_stretched_image = stretch_contrast_of_rgb_image(rgb_image)

plt.subplot(2, 1, 1)
plt.title('Oryginał')
plt.imshow(rgb_image)

plt.subplot(2, 1, 2)
plt.title('Rozciągnięty kontrast')
plt.imshow(contrast_stretched_image)
plt.gcf().set_size_inches((20, 10))

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Oryginalny histogram')
plot_histogram_from_rgb_image(rgb_image)

plt.subplot(2, 1, 2)
plt.title('Histogram z rozciągniętym kontrastem')
plot_histogram_from_rgb_image(contrast_stretched_image)

plt.gcf().set_size_inches((10, 10))

In [ ]:
hist_equalized_r = cv2.equalizeHist(r)
hist_equalized_g = cv2.equalizeHist(g)
hist_equalized_b = cv2.equalizeHist(b)

hist_equalized_image_rgb = cv2.merge((hist_equalized_r, hist_equalized_g, hist_equalized_b))

plt.subplot(2, 1, 1)
plt.title('Orginał')
plt.imshow(rgb_image)

plt.subplot(2, 1, 2)
plt.title('Wyrównany histogram')
plt.imshow(hist_equalized_image_rgb)
plt.gcf().set_size_inches((20, 10))

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Oryginalny histogram')
plot_histogram_from_rgb_image(rgb_image)

plt.subplot(2, 1, 2)
plt.title('Histogram z wyrównaniem')
plot_histogram_from_rgb_image(hist_equalized_image_rgb)

plt.gcf().set_size_inches((10, 10))

Jak widać dla danego przykładowego obrazka rozciągnie kontrastu nic nie robi, natomiast wyrównywanie histogramu zaimplementowane w OpenCV stara się osiągnąć swój cel i jak widać "spłaszcza" histogram. Celem obydwu metod jest rozciągnięcie "górek" na histogramie i o ile dla danego obrazka rozciąganie kontrastu nie daje żadnego wyniku to wyrównywanie po przez usuwanie niektórych intensywności daje dobre wyniki. Obraz jest nieco lepszy wizualnie oraz potencjalnie lepiej przystosowany do dalszej obróbki.

## Progowanie

Przykładowy obrazek w skali szarości

In [ ]:
image = cv2.imread('lab3_1.jpg')

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

plt.imshow(gray_image, cmap='gray')

### Porównanie różnych prostych metod (wymagających podania jakiegoś zadanego progu) binaryzacji obrazu dostępnych w OpenCV.

In [ ]:
methods = [('THRESH_BINARY', cv2.THRESH_BINARY), ('THRESH_BINARY_INV', cv2.THRESH_BINARY_INV), ('THRESH_TRUNC', cv2.THRESH_TRUNC), ('THRESH_TOZERO', cv2.THRESH_TOZERO), ('THRESH_TOZERO_INV', cv2.THRESH_TOZERO_INV)]

thresh_value = 127
plt.subplot(6, 1, 1)
plt.title('Oryginał')
plt.imshow(gray_image, cmap='gray')

for i in range(len(methods)):
    method = methods[i][1]

    _, thresh = cv2.threshold(gray_image, thresh_value, 255, method)
    plt.subplot(6, 1, i + 2)
    plt.title(methods[i][0])
    plt.imshow(thresh, cmap='gray')

plt.gcf().set_size_inches((50, 30))

### Porównanie metod adaptacyjnego doboru progu.

In [ ]:
neighs = [5, 21, 41, 51]

def adaptive_thresh(gray_image, neighs, method, thresh_type):

    for i in range(len(neighs)):
        plt.subplot(len(neighs), 1, i + 1)

        thresh = cv2.adaptiveThreshold(gray_image, 255, method[1], thresh_type, neighs[i], 2)
        plt.title(f'{method[0]}, neigh = {neighs[i]}')
        plt.imshow(thresh, cmap='gray')

In [ ]:
adaptive_thresh(gray_image, neighs, ('ADAPTIVE_THRESH_MEAN_C', cv2.ADAPTIVE_THRESH_MEAN_C), cv2.THRESH_BINARY)

plt.gcf().set_size_inches((30, 20))

In [ ]:
adaptive_thresh(gray_image, neighs, ('ADAPTIVE_THRESH_GAUSSIAN_C', cv2.ADAPTIVE_THRESH_GAUSSIAN_C), cv2.THRESH_BINARY)

plt.gcf().set_size_inches((30, 20))

Jak widać rozmiar sąsiedztwa w obydwu przypadkach powoduje podobne zachowanie. Przy małym sąsiedztwie w niektórych częściach obrazu (np. niebo lub trawa) jest bardzo dużo pojedynczych pikseli szumu, ale krawędzie są dość dobrze widoczne na budynku. Zwiększenie sąsiedztwa niejako odwraca sytuację i redukuje szum, ale krawędzie zaczynają się "zlewać". Subiektywnie metoda oparta o rozkład Gaussa daje nieco lepsze rezultaty.

### Metoda Otsu automatycznego doboru progu.

In [ ]:
ret_otsu, thresh_otsu = cv2.threshold(gray_image, 0, 255, cv2.THRESH_OTSU)

plt.title(f'Wartość progu Otsu = {ret_otsu}')
plt.imshow(thresh_otsu, cmap='gray')

In [ ]:
counts, bins = np.histogram(gray_image, bins=256)
plt.title('Histogram obrazu w skali szarości z naniesionym progiem Otsu')
plt.scatter(x=bins[:-1], y=counts, color='gray', marker='.', label='Poziom intensywności')
plt.axvline(x=ret_otsu, label=f'Próg Otsu = {ret_otsu}', color='black')
plt.legend()

### Klasteryzacja obrazu metoda KMeans dostępną w OpenCV dla różnych wartości K.

In [ ]:
Ks = [2, 4, 8]

second_image = cv2.imread('lab3_2.png')
second_image_rgb = cv2.cvtColor(second_image, cv2.COLOR_BGR2RGB)

second_image_rgb_flat = second_image_rgb.reshape((-1, 3))
second_image_rgb_flat = second_image_rgb_flat.astype(np.float32)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1.0)

plt.subplot(4, 1, 1)
plt.title('Oryginał')
plt.imshow(second_image_rgb)

for i in range(len(Ks)):
    K = Ks[i]
    ret, labels, centers = cv2.kmeans(second_image_rgb_flat, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    centers = centers.astype(np.uint8)

    ret = centers[labels]
    ret = ret.reshape(second_image_rgb.shape)
    plt.subplot(4, 1, i + 2)
    plt.title(f'Klasteryzacja KMeans z K = {K}')
    plt.imshow(ret)

plt.gcf().set_size_inches((50, 30))

Klasteryzacja obrazu ze względu na ilość środków klastrów po 3 składowych kolorach daje oczekiwanych wynik. Środki znajodowane w algorytmie odpowiadają kolorom, które często występują (dominują) w obrazie, jak np. odcienie zielonego. Kolory te odpowiadają pewnym znaczącym elementom/obiektom w obrazie. Jednym z nich jest na przykład układ dróg (kolor szary).

### Implementacja algorytmu Otsu.

In [ ]:
def my_otsu(gray_image):
    counts, bins = np.histogram(gray_image, bins=256)
    bins = bins[:-1]

    normalized_hist = counts / counts.sum()

    fn_max = -np.inf
    ret_my_otsu = -1

    for i in range(1, 255):
        c1_bins, c2_bins = np.arange(0, i), np.arange(i + 1, 256)

        o1, o2 = normalized_hist[:i].sum(), normalized_hist[i + 1:].sum()
        u1, u2 = np.sum(c1_bins * normalized_hist[:i] / o1), np.sum(c2_bins * normalized_hist[i + 1:] / o2)

        fn = o1 * o2 * (u2 - u1)**2

        if fn > fn_max:
            fn_max = fn
            ret_my_otsu = i

    return ret_my_otsu

In [ ]:
ret_my_otsu = my_otsu(gray_image)

_, thresh = cv2.threshold(gray_image, ret_my_otsu, 255, cv2.THRESH_BINARY)
plt.subplot(2, 1, 1)
plt.title(f'Próg Otsu znaleziony własną implementacją = {ret_my_otsu}')
plt.imshow(thresh, cmap='gray')

plt.subplot(2, 1, 2)
plt.title(f'Próg Otsu znaleziony algorytmem w OpenCV = {ret_otsu}')
plt.imshow(thresh_otsu, cmap='gray')

plt.gcf().set_size_inches((20, 10))

Próg nie różni się od bibliotecznej implementacji, ale wydajność jego znajdowania przy pomocy mojej implemtacji prawdopodobnie jest znacznie niższa niż ta dla metody bibliotecznej. Potencjalne różnice, jakie mogą się pojawić, są wynikiem błędów zaokrągleń.

## Interpolacja, filtry

Przykładowy obrazek.

In [ ]:
image = cv2.imread('lab3_1.jpg')

rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(rgb_image)

### Porównanie metod interpolacji dostępnych w OpenCV w kontekście zmieniania rozmiaru obrazu.

In [ ]:
inter_methods = [(cv2.INTER_NEAREST, 'INTER_NEAREST'), (cv2.INTER_LINEAR, 'INTER_LINEAR'), (cv2.INTER_AREA, 'INTER_AREA'), (cv2.INTER_CUBIC, ' INTER_CUBIC'), (cv2.INTER_LANCZOS4, 'INTER_LANCZOS4')]

small_image = cv2.resize(rgb_image, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LANCZOS4)

In [ ]:
# na potrzeby dokładności pikselowej i uniknięcia potencjalnych problemów z sposobem wyświetlania obrazków w jupyterze są one zapisywane na dysku
for i in range(len(inter_methods)):
    method = inter_methods[i][0]
    method_name = inter_methods[i][1]

    bigger_image = cv2.resize(small_image, None, fx=1.5, fy=1.5, interpolation=method)
    bigger_image = cv2.cvtColor(bigger_image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f'{method_name}.png', bigger_image)

Po zmniejszeniu rozmiaru przykładowego obrazu o 50% i zwiększeniu o 50%:
NEAREST - metoda ta powoduje oczywiste artefakty w kształcie kwadratów
LINEAR - metoda ta generuje dość oczywisty efekt rozmazania
AREA - metoda ta również generuje arftefakty w kształcie kwadratów ale w dużo mniejszym stopniu niż metoda NEAREST
CUBIC i LANCZOS4 - podobne rezultaty ze znacznie mniejszą liczbą artefaktów w porównaniu do pozostałych metod

### Działanie filtru uśredniającego na przykładowym obrazie.

In [ ]:
mean_kernels = [5, 10, 15]
mean_filtered = []

for i in range(len(mean_kernels)):
    kernel = mean_kernels[i]
    filtered = cv2.blur(rgb_image, (kernel, kernel))
    mean_filtered.append(filtered)
    plt.subplot(len(mean_kernels), 1, i + 1)
    plt.title(f'Filtr uśredniający, jądro =  ({mean_kernels[i]}, {mean_kernels[i]})')
    plt.imshow(filtered)

plt.gcf().set_size_inches((30, 20))

Jak widać filtr uśredniający ma efekt rozmazujący obraz. Natomiast ma też poważną wadę, gdyż wraz z rosnącym rozmiarem jądra, coraz bardziej jest widoczny efek tzw. "color bandingu", który nie tylko źle wygląda, ale może prowadzić do problemów przy dalszej analizie obrazu.

### Działanie filtru medianowego na przykładowym obrazie.

In [ ]:
median_kernels = [5, 11, 15]
median_filtered = []

for i in range(len(median_kernels)):
    kernel = median_kernels[i]
    filtered = cv2.medianBlur(rgb_image, kernel)
    median_filtered.append(filtered)
    plt.subplot(len(median_kernels), 1, i + 1)
    plt.title(f'Filtr medianowy, jądro = ({median_kernels[i]}, {median_kernels[i]})')
    plt.imshow(filtered)

plt.gcf().set_size_inches((30, 20))

Tak jak w przypadku filtru uśredniającego, został uzyskany efekt rozmycia obrazu. W tym przypadku wraz z rosnącym rozmiarem jądra filtru, detale obrazu zostały kompletnie stracone.

### Działanie filtru gaussowskiego na przykładowym obrazie.

In [ ]:
gauss_filtered = cv2.GaussianBlur(rgb_image, (5, 5), 0)

plt.imshow(gauss_filtered)
plt.gcf().set_size_inches((20, 10))

Filtr gaussowski również uzyskuje efekt rozmycia obrazu. Różnica polega na tym, że nie prowadzi on do strat widocznych w poprzednich dwóch filtrach.

### Binaryzacja przykładowego obrazu po filtracji wspomnianymi filtrami i bez niej.

In [ ]:
mean_filtered_gray = cv2.cvtColor(mean_filtered[0], cv2.COLOR_RGB2GRAY)
ret_otsu, thresh_mean_filtered = cv2.threshold(mean_filtered_gray, 0, 255, cv2.THRESH_OTSU)

median_filtered_gray = cv2.cvtColor(median_filtered[0], cv2.COLOR_RGB2GRAY)
ret_otsu, thresh_median_filtered = cv2.threshold(median_filtered_gray, 0, 255, cv2.THRESH_OTSU)

gauss_filtered_gray = cv2.cvtColor(gauss_filtered, cv2.COLOR_RGB2GRAY)
ret_otsu, thresh_gauss_filtered = cv2.threshold(gauss_filtered_gray, 0, 255, cv2.THRESH_OTSU)

gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
ret_otsu, thresh_gray = cv2.threshold(gray_image, 0, 255, cv2.THRESH_OTSU)

plt.subplot(4, 1, 1)
plt.title('Binaryzacja bez filtrowania')
plt.imshow(thresh_gray, cmap='gray')

plt.subplot(4, 1, 2)
plt.title('Binaryzacja po filtrowaniu uśredniającym')
plt.imshow(thresh_mean_filtered, cmap='gray')

plt.subplot(4, 1, 3)
plt.title('Binaryzacja po filtrowaniu medianowym')
plt.imshow(thresh_median_filtered, cmap='gray')

plt.subplot(4, 1, 4)
plt.title('Binaryzacja po filtrowaniu gaussowskim')
plt.imshow(thresh_gauss_filtered, cmap='gray')

plt.gcf().set_size_inches((30, 20))

Jak widać binaryzacja po filtrowaniu jest znacząco inna. Filtrowanie powoduje usunięcie niektórych detali (np. tekstu widocznego na binaryzacji bez filtra). Dodatkowo można zauważyć, że obraz filtrowany gaussowsko zachowuje najwięcej krawędzi ze wszystkich pokazanych metod filtracji wraz z potencjalnie pożądanym efektem usunięcia niektórych detali.

### Filtry: krzyż Robertsa, Prewitta, Sobela.

In [ ]:
Roberts1 = np.array([[0, 1],
                     [-1, 0]])

Roberts2 = np.array([[1, 0],
                     [0, -1]])

Prewitt1 = np.array([[1, 1, 1],
                     [0, 0, 0],
                     [-1, -1, -1]])

Prewitt2 = np.array([[1, 0, -1],
                     [1, 0, -1],
                     [1, 0, -1]])

Sobel1 = np.array([[1, 2, 1],
                   [0, 0, 0],
                   [-1, -2, -1]])

Sobel2 = np.array([[1, 0, -1],
                   [2, 0, -2],
                   [1, 0, -1]])

In [ ]:
roberts1_image = cv2.filter2D(rgb_image, -1, Roberts1)
roberts2_image = cv2.filter2D(rgb_image, -1, Roberts2)

plt.subplot(2, 1, 1)
plt.title('Roberts')
plt.imshow(roberts1_image)

plt.subplot(2, 1, 2)
plt.imshow(roberts2_image)

plt.gcf().set_size_inches(30, 20)

In [ ]:
prewitt1_image = cv2.filter2D(rgb_image, -1, Prewitt1)
prewitt2_image = cv2.filter2D(rgb_image, -1, Prewitt2)

plt.subplot(2, 1, 1)
plt.title('Prewitt')
plt.imshow(prewitt1_image)

plt.subplot(2, 1, 2)
plt.imshow(prewitt2_image)

plt.gcf().set_size_inches(30, 20)

In [ ]:
sobel1_image = cv2.filter2D(rgb_image, -1, Sobel1)
sobel2_image = cv2.filter2D(rgb_image, -1, Sobel2)

plt.subplot(2, 1, 1)
plt.title('Sobel')
plt.imshow(sobel1_image)

plt.subplot(2, 1, 2)
plt.imshow(sobel2_image)

plt.gcf().set_size_inches(30, 20)

Jak widać podane przykładowe filtry Robertsa, Prewitta i Sobela różnią się między samymi sobą typem krawędzi jakie zanjdują. Czyli pierwszy podany filtr Prewitta znajduje krawędzie poziome, a drugi znajduje krawędzie pionowe.

In [ ]:
img = np.abs(prewitt1_image.astype(np.int32) - sobel1_image.astype(np.int32))

plt.imshow(img)

Dodatkowo warto zauważyć na powyższym obrazku, że dla przykładowego poziomego filtru Prewitta i poziomego filtru Sobela wynik jest bardzo zbliżony. Jak łatwo się domyślić wynika to z faktu, że różnią się one jedynie jedną wagą. Środkowy piksel jest brany z dwa razy większą wagą. Stąd też różnica między tymi wynikami jest bardzo mała.